In [3]:
from bs4 import BeautifulSoup
import requests
import shutil


In [21]:
download_image_url(url="https://www.formula1.com/content/dam/fom-website/drivers/L/LANNOR01_Lando_Norris/lannor01.png.transform/2col/image.png",
                  path="test.png")

In [20]:
def download_image_url(url:str, path:str)->bool:
    """
    Downloads an image from the supplid URL and saves it to path.
    """
    response = requests.get(url)


    file = open(path, "wb")

    file.write(response.content)

    file.close()

In [24]:
r = requests.get('https://www.formula1.com/en/drivers.html')
soup = BeautifulSoup(r.content, 'html.parser')
wrapper = soup.select(".container.listing-items--wrapper.driver.during-season")[0]

drivers = []
for i, driver_el in enumerate(wrapper.select(".listing-item--link")):
    driver_dict = dict()
    driver_dict['link'] = driver_el['href']
    driver_dict['rank'] = driver_el.select(".rank")[0].text
    driver_dict['points'] = driver_el.select(".points")[0].select('.f1-wide--s')[0].text
    driver_dict['first_name'] = driver_el.select(".listing-item--name")[0].select('span')[0].text
    driver_dict['second_name'] = driver_el.select(".listing-item--name")[0].select('span')[1].text
    
    try:
        driver_dict['flag_uri'] = driver_el.select(".country-flag")[0].select('source')[0]['data-srcset']
        
        flag_url = f"./images/{driver_dict['first_name'].lower()}_{driver_dict['second_name'].lower()}_flag.png"
        driver_dict["local_flag_ur"] = flag_url
        download_image_url(
            url=driver_dict['flag_uri'].split(', ')[1][:-3], 
            path=flag_url)
    except IndexError:
        driver_dict["local_flag_ur"] = None
        driver_dict['flag_uri'] = None
    
    driver_dict['team'] = driver_el.select(".listing-item--team")[0].text
    driver_dict['driver_image_uri'] = driver_el.select(".listing-item--image-wrapper")[0].select('source')[0]['data-srcset']
    driver_dict['driver_number_uri'] = driver_el.select(".listing-item--number")[0].select('source')[0]['data-srcset']
    
    # Download the images     
    local_image_uri = f"./images/{driver_dict['first_name'].lower()}_{driver_dict['second_name'].lower()}_image.png"
    download_image_url(
        url=driver_dict['driver_image_uri'].split(', ')[1][:-3], 
        path=local_image_uri)
    driver_dict["local_driver_image_uri"] = local_image_uri
    local_driver_number_uri = f"./images/{driver_dict['first_name'].lower()}_{driver_dict['second_name'].lower()}_number.png"
    download_image_url(
        url=driver_dict['driver_number_uri'].split(', ')[1][:-3], 
        path=local_driver_number_uri)
    driver_dict["local_driver_flag_uri"] = local_driver_number_uri
    

    drivers.append(driver_dict)

In [25]:
import pandas as pd
drivers = pd.DataFrame(drivers)
drivers.to_csv("driver_data_from_f1.com.2021.10.03.csv", index=False)